<a href="https://colab.research.google.com/github/NikunjRathod200/Machine-Learning/blob/main/EEG_Dataset_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import numpy as np
import pandas as pd
from collections import Counter

In [ ]:
within_fp = "/content/drive/MyDrive/within_subject.pkl"
cross_fp = "/content/drive/MyDrive/cross_subject.pkl"

In [ ]:
with open(within_fp,'rb')as l:
  within_fp_data = pickle.load(l)

In [ ]:
with open(cross_fp,'rb')as f:
  cross_fp_data = pickle.load(f)

In [ ]:
x_train =within_fp_data['train']['x']
x_train = np.mean(x_train,axis=2)
x_test = within_fp_data['test']['x']
x_test = np.mean(x_test,axis=2)
x_val = within_fp_data['val']['x']
x_val = np.mean(x_val,axis=2)

In [ ]:
X_train = cross_fp_data['train']['x']
X_train = np.mean(X_train,axis=2)
X_test = cross_fp_data['test']['x']
X_test = np.mean(X_test,axis=2)
X_val = cross_fp_data['val']['x']
X_val = np.mean(X_val,axis=2)

In [ ]:
#  a = [[[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5]],
#       [[3,4,5,6,7],[3,4,5,6,7],[3,4,5,6,7]]]
# a = np.array(a)
# a.shape

(2, 3, 5)

In [ ]:
y_train = within_fp_data['train']['y_med']
y_test = within_fp_data['test']['y_med']
y_val = within_fp_data['val']['y_med']

In [ ]:
Y_train = cross_fp_data['train']['y_med']
Y_test = cross_fp_data['test']['y_med']
Y_val = cross_fp_data['val']['y_med']

In [ ]:
X_tr = pd.DataFrame(X_train)
Y_tr = pd.DataFrame(Y_train)
X_ts = pd.DataFrame(X_test)
Y_ts = pd.DataFrame(Y_test)
X_vl = pd.DataFrame(X_val)
Y_vl = pd.DataFrame(Y_val)

In [ ]:
x_tr = pd.DataFrame(x_train)
y_tr = pd.DataFrame(y_train)
x_ts = pd.DataFrame(x_test)
y_ts = pd.DataFrame(y_test)
x_vl = pd.DataFrame(x_val)
y_vl = pd.DataFrame(y_val)

In [ ]:
class PCA:
    def __init__(self, x):
        self.principal_axis = None
        self.mean = np.mean(x, axis=0)

    def fit(self, x, n_components=2):
        # 1. Recenter the data X around mean = 0
        x_mean = np.mean(x, axis=0)
        mean_centred_X = x - x_mean
        # 2. Calculate covarience, cov = (1/m) X^T . X
        x = mean_centred_X.T
        cov_matrix = np.dot(x, x.T) * (1/np.shape(x)[0])
        # 3. Calculate eigenvalue and eigenvector of the cov_matrix.
        # Hint: Use np.linalg.eigh()
        eg_vals, eg_vecs = np.linalg.eig(cov_matrix)
        # 4. Sort eigenvalues & arrange eigenvectors according to sorted eigenvalues
        sorted_indices_eg_val = np.argsort(eg_vals)[::-1]
        sorted_eg_vecs = eg_vecs[sorted_indices_eg_val]
        # 5. Select n_components from sorted_eg_vecs.
        # These principal components will be used for reduction in self.transform()
        self.principal_axis = sorted_eg_vecs[0:n_components]
        return
    def transform(self, x):
        x = x - self.mean
        projected_data = np.dot(x, self.principal_axis.T)
        return projected_data

In [ ]:
pcaObj = PCA(x_tr)
pcaObj.fit(x_tr,4)
x_tr=pd.DataFrame(pcaObj.transform(x_tr))

pcaObj = PCA(x_ts)
pcaObj.fit(x_ts,4)
x_ts=pd.DataFrame(pcaObj.transform(x_ts))

pcaObj = PCA(x_vl)
pcaObj.fit(x_vl,4)
x_vl=pd.DataFrame(pcaObj.transform(x_vl))

# Random Forest from Sklearn

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score
rfc = RandomForestClassifier()

In [ ]:
rfc.fit(x_tr,y_tr)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier()

In [ ]:
y_pred_train = rfc.predict(x_tr)
y_pred_val = rfc.predict(x_vl)
y_pred_test = rfc.predict(x_ts)

In [ ]:
rfc.fit(X_tr,Y_tr)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier()

In [ ]:
Y_pred_train = rfc.predict(X_tr)
Y_pred_val = rfc.predict(X_vl)
Y_pred_test = rfc.predict(X_ts)

##Test Accuracy , Train Accuracy , Val Accuracy for RandomForest

Within Data --> (0.9971264367816092, 1.0, 1.0)

Cross Data -->  (0.342668863261944, 1.0, 0.2289348171701113)

In [ ]:
accuracy_score(y_ts,y_pred_test),accuracy_score(y_tr,y_pred_train),accuracy_score(y_vl,y_pred_val)

(0.21408045977011494, 1.0, 0.26777609682299547)

In [ ]:
accuracy_score(Y_ts,Y_pred_test),accuracy_score(Y_tr,Y_pred_train),accuracy_score(Y_vl,Y_pred_val)

(0.21087314662273476, 1.0, 0.19395866454689983)

# F1 score for RandomForest Classifier

In [ ]:
f1_score(y_ts,y_pred_test,average='micro')

0.9971264367816092

In [ ]:
f1_score(Y_ts,Y_pred_test,average='macro')

0.16966752947444008

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
params = {'n_estimators':[100,200,300,500,1000],
                  'criterion':['gini','entropy'],
                  'max_depth': [2,3]}
rcv = RandomizedSearchCV(rfc,params)

In [ ]:
rcv.fit(X_tr,Y_tr)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 

RandomizedSearchCV(estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [2, 3],
                                        'n_estimators': [100, 200, 300, 500,
                                                         1000]})

In [ ]:
rcv.best_params_

{'n_estimators': 1000, 'max_depth': 3, 'criterion': 'entropy'}

In [ ]:
rcv.best_score_

0.9899977270144336

In [ ]:
Y_pred_test = rcv.best_estimator_.predict(X_ts)
Y_pred_val = rcv.best_estimator_.predict(X_vl)

# Increased Accuracy

In [ ]:
accuracy_score(Y_pred_test,Y_ts),accuracy_score(Y_pred_val,Y_vl)

(0.19110378912685339, 0.29411764705882354)

In [ ]:
f1_score(Y_ts,Y_pred_test,average='micro')

0.19110378912685339

# Knn from Sklearn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
knn.fit(x_tr,y_tr)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [ ]:
knn.fit(X_tr,Y_tr)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [ ]:
y_pred_train = knn.predict(x_tr)
y_pred_val = knn.predict(x_vl)
y_pred_test = knn.predict(x_ts)

In [ ]:
Y_pred_train = knn.predict(X_tr)
Y_pred_vat = knn.predict(X_vl)
Y_pred_test = knn.predict(X_ts)

#  Train Accuracy , Val Accuracy,Test Accuracy for KNN

Within Data --> (0.9989888776541962, 0.9984871406959153, 0.9971264367816092)

Cross Data --> (1.0, 0.13036565977742448, 0.2602965403624382)

In [ ]:
accuracy_score(y_tr,y_pred_train),accuracy_score(y_vl,y_pred_val),accuracy_score(y_ts,y_pred_test)

(0.9878665318503539, 0.27685325264750377, 0.2557471264367816)

In [ ]:
accuracy_score(Y_tr,Y_pred_train),accuracy_score(Y_vl,Y_pred_vat),accuracy_score(Y_ts,Y_pred_test)

(1.0, 0.13036565977742448, 0.2602965403624382)

# F1 score for Knn

In [ ]:
f1_score(Y_ts,Y_pred_test,average='micro')

0.2602965403624382

In [ ]:
f1_score(y_ts,y_pred_test,average='micro')

0.9971264367816092

In [ ]:
params = {'n_neighbors':[30,40,50,100,150],'weights':['distance','uniform'],'algorithm':['auto','ball_tree','kd_tree','brute'],'leaf_size':[30,40,50,60,70],'p':[1,2]}
rcv = RandomizedSearchCV(knn,params)
rcv.fit(X_tr,Y_tr)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exam

RandomizedSearchCV(estimator=KNeighborsClassifier(),
                   param_distributions={'algorithm': ['auto', 'ball_tree',
                                                      'kd_tree', 'brute'],
                                        'leaf_size': [30, 40, 50, 60, 70],
                                        'n_neighbors': [30, 40, 50, 100, 150],
                                        'p': [1, 2],
                                        'weights': ['distance', 'uniform']})

In [ ]:
rcv.best_params_

{'weights': 'distance',
 'p': 2,
 'n_neighbors': 50,
 'leaf_size': 60,
 'algorithm': 'auto'}

In [ ]:
rcv.best_score_

0.9985714285714286

In [ ]:
Y_pred_test = rcv.best_estimator_.predict(X_ts)
Y_pred_val = rcv.best_estimator_.predict(X_vl)

In [ ]:
accuracy_score(Y_pred_test,Y_ts),accuracy_score(Y_pred_val,Y_vl)

(0.2586490939044481, 0.1287758346581876)

In [ ]:
f1_score(Y_pred_test,Y_ts,average='micro')

0.2586490939044481

# Naive Bayes from Sklearn

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [ ]:
gnb.fit(x_tr,y_tr)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [ ]:
gnb.fit(X_tr,Y_tr)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [ ]:
Y_pred_train = gnb.predict(X_tr)
Y_pred_val = gnb.predict(X_vl)
Y_pred_test = gnb.predict(X_ts)

In [ ]:
y_pred_train = gnb.predict(x_tr)
y_pred_val = gnb.predict(x_vl)
y_pred_test = gnb.predict(x_ts)

#Train Accuracy , Val Accuracy,Test Accuracy Naive Bayes

Within Data --> (0.48432760364004046, 0.49319213313161875, 0.49712643678160917)

Croff Data --> (0.4187708432586946, 0.2527821939586645, 0.500823723228995)

In [ ]:
accuracy_score(y_tr,y_pred_train),accuracy_score(y_vl,y_pred_val),accuracy_score(y_ts,y_pred_test)

(0.48432760364004046, 0.49319213313161875, 0.49712643678160917)

In [ ]:
accuracy_score(Y_tr,Y_pred_train),accuracy_score(Y_vl,Y_pred_val),accuracy_score(Y_ts,Y_pred_test)

(0.49928537398761313, 0.33068362480127184, 0.20922570016474465)

# F1 score for Naive Bayes

In [ ]:
f1_score(y_ts,y_pred_test,average='micro')

0.49712643678160917

In [ ]:
f1_score(Y_ts,Y_pred_test,average='micro')

0.20922570016474465

# Logistic Regression from sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(multi_class='multinomial',max_iter=10000)

In [ ]:
lr.fit(x_tr,y_tr)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=10000, multi_class='multinomial')

In [ ]:
y_pred_train = lr.predict(x_tr)
y_pred_val = lr.predict(x_vl)
y_pred_test = lr.predict(x_ts)

In [ ]:
lr.fit(X_tr,Y_tr)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=10000, multi_class='multinomial')

In [ ]:
Y_pred_train = lr.predict(X_tr)
Y_pred_val = lr.predict(X_vl)
Y_pred_test = lr.predict(X_ts)

# Train Accuracy , Val Accuracy,Test Accuracy for Logistic Regression

Within Data --> (1.0, 0.9984871406959153, 0.9942528735632183)

Croff Data --> (1.0, 0.35135135135135137, 0.24382207578253706)

In [ ]:
accuracy_score(y_tr,y_pred_train),accuracy_score(y_vl,y_pred_val),accuracy_score(y_ts,y_pred_test)

(1.0, 0.9984871406959153, 0.9942528735632183)

In [ ]:
accuracy_score(Y_tr,Y_pred_train),accuracy_score(Y_vl,Y_pred_val),accuracy_score(Y_ts,Y_pred_test)

(1.0, 0.35135135135135137, 0.24382207578253706)

# F1 score for Logistic Regression

In [ ]:
f1_score(y_ts,y_pred_test,average='micro')

0.9942528735632183

In [ ]:
f1_score(Y_ts,Y_pred_test,average='micro')

0.24382207578253706

# Support Vector Classifier from sklearn

In [ ]:
from sklearn.svm import SVC
svc =SVC(max_iter=1000,kernel='poly')

In [ ]:
svc.fit(x_tr,y_tr)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


SVC(kernel='poly', max_iter=1000)

In [ ]:
svc.fit(X_tr,Y_tr)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='poly', max_iter=1000)

In [ ]:
Y_pred_train = svc.predict(X_tr)
Y_pred_val = svc.predict(X_vl)
Y_pred_test = svc.predict(X_ts)

In [ ]:
y_pred_train = svc.predict(x_tr)
y_pred_val = svc.predict(x_vl)
y_pred_test = svc.predict(x_ts)

# Train Accuracy , Val Accuracy,Test Accuracy for Logistic Regression

Within Data --> (0.7785642062689585, 0.7821482602118003, 0.7816091954022989)

Croff Data --> (0.9137684611719866, 0.17647058823529413, 0.4596375617792422)

In [ ]:
accuracy_score(y_tr,y_pred_train),accuracy_score(y_vl,y_pred_val),accuracy_score(y_ts,y_pred_test)

(0.7785642062689585, 0.7821482602118003, 0.7816091954022989)

In [ ]:
accuracy_score(Y_tr,Y_pred_train),accuracy_score(Y_vl,Y_pred_val),accuracy_score(Y_ts,Y_pred_test)

(0.9137684611719866, 0.17647058823529413, 0.4596375617792422)

# F1 score for SVM

In [ ]:
f1_score(Y_ts,Y_pred_test,average='micro')

0.4596375617792422

In [ ]:
f1_score(y_ts,y_pred_test,average='micro')

0.781609195402299

# Gradient Boost from Sklearn

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb =GradientBoostingClassifier()

In [ ]:
gb.fit(x_tr,y_tr)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier()

In [ ]:
y_pred_train = gb.predict(x_tr)
y_pred_val = gb.predict(x_vl)
y_pred_test = gb.predict(x_ts)

In [ ]:
gb.fit(X_tr,Y_tr)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier()

In [ ]:
Y_pred_train = gb.predict(X_tr)
Y_pred_val = gb.predict(X_vl)
Y_pred_test = gb.predict(X_ts)

# Train Accuracy , Val Accuracy,Test Accuracy for Gradient Boost

Within Data --> (1.0, 0.9969742813918305, 0.9985632183908046)

Croff Data --> 1.0, 0.39109697933227344, 0.2751235584843493)

In [ ]:
accuracy_score(Y_tr,Y_pred_train),accuracy_score(Y_vl,Y_pred_val),accuracy_score(Y_ts,Y_pred_test)

(1.0, 0.39109697933227344, 0.2751235584843493)

In [ ]:
accuracy_score(y_tr,y_pred_train),accuracy_score(y_vl,y_pred_val),accuracy_score(y_ts,y_pred_test)

(1.0, 0.9969742813918305, 0.9985632183908046)

# F1 score for Gradient Boost

In [ ]:
f1_score(y_ts,y_pred_test,average='micro')

0.9985632183908046

In [ ]:
f1_score(Y_ts,Y_pred_test,average='micro')

0.2751235584843493